In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset = pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [4]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [5]:
dataset.drop('User ID',axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [6]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
indep = dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep = dataset['Purchased']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep, dep, test_size=1/3,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'], 'metric':['minkowski'],
              'weights':['uniform', 'distance']}
grid = GridSearchCV (KNeighborsClassifier(), param_grid, refit = True, verbose = 3, n_jobs = -1, scoring = 'f1_weighted')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski'],
                         'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [13]:
# finding the best parameter
#print(grid.best_params_)
re = grid.cv_results_
grid_predictions = grid.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,grid_predictions)

In [15]:
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test,grid_predictions)

In [16]:
from sklearn.metrics import f1_score
f1_macro = f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter{'algorithm': 'auto', 'metric': 'minkowski', 'weights': 'uniform'}: 0.9183922682195829


In [17]:
print("The Confusion Matrix:\n",cm)

The Confusion Matrix:
 [[78  7]
 [ 4 45]]


In [18]:
print("classification_report:\n",clf_report)

classification_report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93        85
           1       0.87      0.92      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.92      0.91       134
weighted avg       0.92      0.92      0.92       134



In [19]:
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003998,0.000003,0.032198,0.008932,auto,minkowski,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.867478,0.906252,0.869709,0.944161,0.981233,0.913767,0.043854,1
1,0.006001,0.004049,0.023197,0.009151,auto,minkowski,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.847141,0.886792,0.851527,0.944161,0.962264,0.898377,0.047185,5
2,0.003799,0.001166,0.029798,0.004262,ball_tree,minkowski,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.867478,0.906252,0.869709,0.944161,0.981233,0.913767,0.043854,1
3,0.004932,0.001676,0.014468,0.005905,ball_tree,minkowski,distance,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.847141,0.886792,0.851527,0.944161,0.962264,0.898377,0.047185,5
4,0.008799,0.007440,0.030797,0.001719,kd_tree,minkowski,uniform,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.867478,0.906252,0.869709,0.944161,0.981233,0.913767,0.043854,1
5,0.004400,0.000801,0.012399,0.002499,kd_tree,minkowski,distance,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.847141,0.886792,0.851527,0.944161,0.962264,0.898377,0.047185,5
6,0.002799,0.000400,0.010199,0.000979,brute,minkowski,uniform,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.867478,0.906252,0.869709,0.944161,0.981233,0.913767,0.043854,1
7,0.003202,0.000753,0.008196,0.001476,brute,minkowski,distance,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.847141,0.886792,0.851527,0.944161,0.962264,0.898377,0.047185,5


In [21]:
age_input = float(input("Age:"))
salary_input = float(input("Salary:"))
sex_male_input = int(input("sex_male 0 or 1:"))
Future_Prediction = grid.predict([[age_input, salary_input, sex_male_input]])
Future_Prediction

Age: 23
Salary: 1221323
sex_male 0 or 1: 0


array([1], dtype=int64)